In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hlsf.models import *
from astropy.io import fits
from astropy import wcs
%matplotlib qt

In [2]:
config = 'H'
lamp = 'linspace256'

In [3]:
if lamp == 'linspace256':
    file_listLines = '../exposures/line_catalog_linspace256.fits'
else:
    file_listLines = f'../text/{lamp}.txt'

In [4]:
lsf_data = LSF_DATA(f"../exposures/ARC-{lamp}_CLEAR_20MAS_{config}_PRM.fits", file_listLines, f"../exposures/WAVECAL_TABLE_20MAS_{config}.fits", 
                        f"../exposures/SLITLET_TABLE_20MAS_{config}.fits", slice=0, normal='Normal')
lsf_data.get_line_list()

{11: 14152.745098039213,
 12: 14171.176470588232,
 13: 14189.607843137252,
 14: 14208.03921568627,
 15: 14226.470588235292,
 16: 14244.901960784311,
 17: 14263.333333333332,
 18: 14281.76470588235,
 19: 14300.196078431369,
 20: 14318.62745098039,
 21: 14337.058823529409,
 22: 14355.490196078428,
 23: 14373.92156862745,
 24: 14392.352941176467,
 25: 14410.784313725488,
 26: 14429.215686274507,
 27: 14447.647058823526,
 28: 14466.078431372547,
 29: 14484.509803921565,
 30: 14502.941176470586,
 31: 14521.372549019605,
 32: 14539.803921568624,
 33: 14558.235294117645,
 34: 14576.666666666664,
 35: 14595.098039215682,
 36: 14613.529411764703,
 37: 14631.960784313722,
 38: 14650.392156862743,
 39: 14668.823529411762,
 40: 14687.254901960781,
 41: 14705.6862745098,
 42: 14724.11764705882,
 43: 14742.54901960784,
 44: 14760.98039215686,
 45: 14779.41176470588,
 46: 14797.8431372549,
 47: 14816.274509803918,
 48: 14834.705882352939,
 49: 14853.137254901958,
 50: 14871.568627450977,
 51: 14889.9

In [9]:
mod = LSF_MODEL.from_json('../file/gauss_hermite_model_2_H_Ar.json')
mod1 = LSF_MODEL.from_json('../file/gaussian_model_2_H_Ar.json')

Destructor called


In [10]:
fig = plt.figure()
ax = plt.axes()
plt.xlabel(r"wavelength ($\AA$)")
plt.ylabel("RMS error")
mod.plot_error_rms(lsf_data, ax)
mod1.plot_error_rms(lsf_data, ax)
plt.legend(['GH_model', 'Gaussian_model'])
plt.grid()
plt.title(f"Comparaison config {lsf_data.config}\nfrom line {lsf_data._lineUp} to line {lsf_data._lineDown}", fontweight='bold')
plt.show()

In [15]:
hdul = fits.open('../images/image_HK.fits')
image = hdul['INTENSITY'].data

In [18]:
import warnings
with warnings.catch_warnings():
    # Ignore a warning on using DATE-OBS in place of MJD-OBS
    warnings.filterwarnings('ignore', message="'datfix' made the change",
                            category=wcs.FITSFixedWarning)
    wcs = wcs.WCS(hdul['INTENSITY'].header)

In [19]:
fig = plt.figure()
fig.add_subplot(111, projection=wcs)
plt.xlabel(r'relative position ($\AA$)')
plt.ylabel(r'wavelength of lines ($\AA$)')
plt.imshow(image, origin='lower', cmap=plt.cm.viridis)
plt.show()